In [1]:
#安放路径
import os

# BASE_PATH = '../data'
BASE_PATH = r'C:\Users\ljs11\Desktop\实验设计作业\Final_code\data'
# BASE_PATH = '/Users/lijiale/Desktop/FinTech2020-master(爸爸方案)2/Final_code/data'

RAW_DATA_PATH = os.path.join(BASE_PATH, 'RawData')
TEMP_DATA_PATH = os.path.join(BASE_PATH, "TempData")
ETL_DATA_PATH = os.path.join(BASE_PATH, 'EtlData')
RESULT_PATH = os.path.join(BASE_PATH, "Result")

SUBMISSION = os.path.join(RESULT_PATH, 'submission.txt')
########################################################################################################################


In [2]:
#读取总数据表
import numpy as np
import pandas as pd

real_data=pd.read_csv(r'C:\Users\ljs11\Desktop\实验设计作业\Final_code\data\real_data.csv')
########################################################################################################################


In [3]:
import pandas as pd
from parameter import *
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold

# os.system("python 2_0lgb.py")
# 删除某一类别占比超过99%的列
good_cols = list(real_data.columns)
for col in real_data.columns:
    rate = real_data[col].value_counts(normalize=True, dropna=False).values[0]
    if rate > 0.999:
        good_cols.remove(col)
        print(col, rate)

real_data = real_data[good_cols]

train = real_data[real_data['TrainOrTest'] == 1]
test = real_data[real_data['TrainOrTest'] == 0]

submission = pd.DataFrame()
submission['id'] = test['id']

params_lgb = {
        'boosting_type': 'gbdt',
        'n_estimators': 1000,
        'max_depth': -1,
        'objective': 'binary',
        'metric': {'auc'},
        'num_leaves': 16,
        'learning_rate': 0.1,
        'feature_fraction': 1.,
        'bagging_fraction': 1.,
        'reg_lambda': 0.5,
        'reg_alpha': 0.3,
        'random_state': 1024,
        'n_jobs': -1,
    }

drop_fea = ['id', 'flag', 'isTest','TrainOrTest']
    # features = [col for col in test.columns if col not in drop_fea]
    # TOP100
features = ['cur_credit_min_opn_dt_cnt', 'l1y_crd_card_csm_amt_dlm_cd', 'Dat_Flg1_Cd_per_count_amt_1',
                'perm_crd_lmt_cd',
                'Trx_Cod2_Cd54_recent_trd',
                # 'cny_trx_amt_sum_6',
                'Dat_Flg3_Cd1_recent_trd', 'acdm_deg_cd',
                'cur_debit_min_opn_dt_cnt', 'Dat_Flg3_Cd_per_count_amt_1', 'Trx_Cod1_Cd_amt_2',
                'Dat_Flg1_Cd_per_count_amt_0',
                'Trx_Cod1_Cd_per_count_amt_2', 'Trx_Cod1_Cd_per_day_times_2', 'Trx_Cod1_Cd_amt_0', 'gdr_cd',
                'Trx_Cod2_Cd_per_count_amt_36', 'pot_ast_lvl_cd', 'job_year',
                # 'cny_trx_amt_sum_5',
                'hld_crd_card_grd_cd',
                'Trx_Cod1_Cd_per_count_amt_0', 'fr_or_sh_ind', 'age', '6_Dat_Flg3_Cd_times_1',
                'Dat_Flg1_Cd_per_day_times_1',
                'Trx_Cod1_Cd1_recent_trd', 'Trx_Cod2_Cd_per_day_times_2', 'Trx_Cod2_Cd_amt_36',
                'Dat_Flg1_Cd5_per_day_times_0',
                'per_day_trd_amt', 'Trx_Cod2_Cd53_recent_trd', 'Trx_Cod2_Cd5_per_day_times_26', 'his_lng_ovd_day',
                'hav_car_grp_ind', 'Dat_Flg1_Cd5_per_day_times_1',
                # 'Trx_Cod2_Cd1_recent_trd',
                'Dat_Flg3_Cd_per_count_amt_0',
                # 'trd_amt',
                'dnl_bind_cmb_lif_ind', 'Trx_Cod2_Cd6_per_day_times_32', 'Trx_Cod2_Cd6_per_day_times_54',
                'Trx_Cod2_Cd_amt_54', 'Dat_Flg1_Cd6_per_day_times_1', 'Trx_Cod2_Cd_per_count_amt_30',
                'Trx_Cod2_Cd6_per_day_times_2', 'Dat_Flg3_Cd_amt_1', 'per_count_trd_amt',
                'Trx_Cod2_Cd_per_day_times_46',
                'Dat_Flg3_Cd6_per_day_times_1',
                # 'Trx_Cod1_Cd2_recent_trd',
                'frs_agn_dt_cnt', 'edu_deg_cd',
                'Trx_Cod2_Cd_per_day_times_15', 'dnl_mbl_bnk_ind', 'Trx_Cod2_Cd29_recent_trd', 'Dat_Flg1_Cd_amt_1',
                'Trx_Cod2_Cd_amt_32', 'Trx_Cod2_Cd_per_count_amt_55', 'crd_card_act_ind',
                'Trx_Cod2_Cd_per_count_amt_32',
                'Dat_Flg3_Cd_per_day_times_1', 'Trx_Cod2_Cd_amt_53', 'Trx_Cod2_Cd32_recent_trd', 'Dat_Flg1_Cd_amt_0',
                'Trx_Cod2_Cd5_per_day_times_32', 'per_day_trd_times', 'Trx_Cod2_Cd_amt_55', 'Trx_Cod2_Cd_amt_30',
                'Trx_Cod2_Cd_amt_52', '5_Trx_Cod2_Cd_times_54', 'Trx_Cod2_Cd_per_day_times_40',
                'Trx_Cod2_Cd_per_count_amt_54',
                'Trx_Cod2_Cd_per_day_times_3', 'Trx_Cod1_Cd5_per_day_times_2', 'Dat_Flg3_Cd_times_1',
                'Trx_Cod2_Cd_per_day_times_54', 'Dat_Flg3_Cd5_per_day_times_1', '6_Trx_Cod2_Cd_times_10',
                'Trx_Cod2_Cd6_per_day_times_1', 'Trx_Cod2_Cd5_per_day_times_30', 'Trx_Cod2_Cd_per_count_amt_11',
                'Trx_Cod2_Cd_amt_24', 'Dat_Flg3_Cd_amt_0', 'Trx_Cod1_Cd6_per_day_times_2',
                # 'Trx_Cod2_Cd30_recent_trd',
                'Trx_Cod2_Cd_per_day_times_16', '6_Trx_Cod2_Cd_times_54', 'Trx_Cod2_Cd6_per_day_times_55',
                'Trx_Cod2_Cd_per_day_times_32',
                # 'Trx_Cod1_Cd0_recent_trd',
                'Trx_Cod2_Cd_per_day_times_0',
                'Trx_Cod1_Cd_per_day_times_1', 'Trx_Cod1_Cd_per_count_amt_1', 'Trx_Cod2_Cd_per_count_amt_21',
                'Trx_Cod2_Cd_times_54', 'Dat_Flg1_Cd_per_day_times_0', 'Trx_Cod2_Cd5_per_day_times_54',
                # 'Trx_Cod2_Cd16_recent_trd',
                'Dat_Flg1_Cd0_recent_trd']
train_X = train[features]
train_y = train['flag']

print(train_X.shape, test[features].shape)

def lgb_model(train_X, label, test, params_lgb):
    fea_dict = {v: k for k, v in enumerate(train_X.columns)}
    train_X.columns = [fea_dict[v] for v in train_X.columns]
    test.columns = train_X.columns

    fea_dict = {v: k for k, v in fea_dict.items()}
    # lgb 模型
    cv_pred = np.zeros(test.shape[0])
    cv_best_auc_all = 0

    # SKF = StratifiedKFold(n_splits=5, shuffle=True, random_state=2019)#5折交叉验证
    SKF = StratifiedKFold(n_splits=10, shuffle=True, random_state=2019)#10折交叉验证
    # SKF = StratifiedKFold(n_splits=20, shuffle=True, random_state=2019)#20折交叉验证
    # SKF = StratifiedKFold(n_splits=30, shuffle=True, random_state=2019)#30折交叉验证

    skf = SKF.split(train_X, label)

    fea_importances = pd.DataFrame({'column': train_X.columns})

    stack_train = np.zeros((train.shape[0], 1))
    stack_test = np.zeros((test.shape[0], 1))

    for i, (train_fold, validate) in enumerate(skf):
        print("model: lgb. fold: ", i, "training...")
        X_train, label_train = train_X.iloc[train_fold], label.iloc[train_fold]
        X_validate, label_validate = train_X.iloc[validate], label.iloc[validate]

        # print(X_train.shape, test.shape)
        dtrain = lgb.Dataset(X_train, label_train)
        dvalid = lgb.Dataset(X_validate, label_validate, reference=dtrain)

        bst = lgb.train(params_lgb, dtrain, valid_sets=(dtrain, dvalid),
                        verbose_eval=50, early_stopping_rounds=100)
        cv_pred += bst.predict(test, num_iteration=bst.best_iteration)
        cv_best_auc_all += bst.best_score['valid_1']['auc']

        score_va = bst.predict(train_X.iloc[validate], num_iteration=bst.best_iteration)
        score_te = bst.predict(test, num_iteration=bst.best_iteration)
        stack_train[validate] += score_va[:, None]
        stack_test += score_te[:, None]

        fea_importance_temp = pd.DataFrame({
            'column': train_X.columns,
            'importance_' + str(i): bst.feature_importance(importance_type='split', iteration=bst.best_iteration)
        })

        fea_importances = fea_importances.merge(fea_importance_temp, how='left', on='column')

    cv_pred /= 5
    cv_best_auc_all /= 5
    # valid_best_auc_all /= 5
    print("lgb cv score for valid is: ", cv_best_auc_all)
    # print("lgb valid score for valid is: ", valid_best_auc_all)

    fea_importances['importance'] = (fea_importances['importance_0'] + fea_importances['importance_1'] +
                                     fea_importances['importance_2'] + fea_importances['importance_3'] +
                                     fea_importances['importance_4']) / 5

    fea_importances = fea_importances[['column', 'importance']]
    fea_importances = fea_importances.sort_values(by='importance', ascending=False)
    fea_importances['column'] = fea_importances['column'].apply(lambda x: fea_dict[x])

    stack_test /= 5
    stack = np.vstack([stack_train, stack_test])
    df_stack = pd.DataFrame()

    df_stack['lgb_prob'] = np.around(stack[:, 0], 6)
    return cv_pred, fea_importances, df_stack

result, feature_importance, df_stack = lgb_model(train_X, train_y, test[features], params_lgb)

print(train_X.shape, test[features].shape)

print('\n most importance features: ')
print(list(feature_importance.iloc[:150]['column']))
print('\n features(importance=0): ')
print(list(feature_importance[feature_importance['importance'] == 0]['column']))

# saving result
print('saving result...')
submission['pred'] = result
submission.to_csv(r'C:\Users\ljs11\Desktop\实验设计作业\Final_code\data\Result\LGB改进_结果.csv', index=False, header=0, encoding='utf-8')
print('done')


isTest 1.0
Trx_Cod2_Cd_times_20 1.0
Trx_Cod2_Cd_times_43 0.9999749517821807
Trx_Cod2_Cd_times_56 0.9993487463366981
Trx_Cod2_Cd_per_day_times_20 1.0
Trx_Cod2_Cd_per_day_times_43 0.9999749517821807
Trx_Cod2_Cd_per_day_times_56 0.9993487463366981
Trx_Cod2_Cd_amt_20 1.0
Trx_Cod2_Cd_amt_43 0.9999749517821807
Trx_Cod2_Cd_amt_56 0.999423890990156
Trx_Cod2_Cd_per_count_amt_20 1.0
Trx_Cod2_Cd_per_count_amt_43 0.9999749517821807
Trx_Cod2_Cd_per_count_amt_56 0.999423890990156
5_Trx_Cod2_Cd_times_20 1.0
5_Trx_Cod2_Cd_times_44 0.9995741802970719
5_Trx_Cod2_Cd_times_50 0.9991483605941437
5_Trx_Cod2_Cd_times_56 0.999499035643614
Trx_Cod2_Cd5_per_day_times_20 1.0
Trx_Cod2_Cd5_per_day_times_44 0.9995741802970719
Trx_Cod2_Cd5_per_day_times_50 0.9991483605941437
Trx_Cod2_Cd5_per_day_times_56 0.999499035643614
6_Trx_Cod2_Cd_times_20 1.0
6_Trx_Cod2_Cd_times_43 0.9999749517821807
6_Trx_Cod2_Cd_times_50 0.999173408811963
6_Trx_Cod2_Cd_times_56 0.9996994213861684
Trx_Cod2_Cd6_per_day_times_20 1.0
Trx_Cod2_Cd

C:\Users\ljs11\Anaconda3\envs\ljs_opencv\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
